<i><b>Public AI</b></i>
<br>

###  &nbsp;&nbsp; **✎&nbsp;&nbsp;Week 2. CNN Basis**
# Section 5. 5_합성곱_신경망의_층 구성하기

### _Objective_
1. 합성곱 층은 이전에 배운 DNN와 비슷하게, Logit을 계산하는 부분과 활성화 함수를 계산하는 부분으로 나누어집니다. <br>
2. 고전 CNN 모델을 살펴보면서, CNN의 구조에 대해 배워보도록 하겠습니다.  <br> 
  


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

<br><br>

# \[ 1. 합성곱 층의 구조 \]

----
----
 
> *합성곱 연산도 이전의 Fully Connected Layed와 마찬가지로, Bias가 존재합니다.*<br>
> *합성곱 연산 이후 활성화 함수를 넣음으로써, 비선형성을 증대시킵니다.*<br>

## 1. 합성곱 연산에 Bias 추가하기
---

* 각 필터 별로 Bias를 추가함으로써, 필터 별 Threshold을 학습할 수 있게 됩니다.<br>

### (1) 3차원 데이터 구성하기

In [ ]:
in0 = np.array([
    [1,4,2,0],
    [2,3,1,1],
    [3,1,3,2],
    [4,2,1,4],
])
in1 = np.array([
    [1,7,0,2],
    [3,2,1,1],
    [0,7,5,1],
    [0,1,5,2],    
])
in2 = np.array([
    [1,5,1,5],
    [3,2,1,9],
    [4,2,2,7],
    [1,2,1,9],
])
inputs = np.stack([in0, in1, in2],axis=-1)
print("입력값의 형태 (H, W, C) :({},{},{})".format(*inputs.shape))

### (2) Filter 와 Bias 구성하기
* 하나의 유닛당 하나의 bias 가 존재 합니다. 

* filter 는 하나의 유닛이라 생각할 수 있습니다. 즉 bias 는 filter 의 갯수만큼 생성됩니다.



In [ ]:
filter_1 = np.array([
    [[2,0,1],[0,1,2],[1,0,2]],
    [[0,1,3],[2,1,3],[4,1,2]],
    [[3,2,1],[2,2,3],[0,0,1]]])
filter_2 = np.array([
    [[4,0,1],[0,0,4],[0,3,2]],
    [[6,1,2],[3,5,1],[2,3,2]],
    [[1,4,1],[1,3,1],[2,1,0]]])

filters = np.stack([filter_1,filter_2],axis=0)
print("Filter의 형태 (N, H, W, C) : ({},{},{},{})".format(*filters.shape))

In [ ]:
bias = np.array([-100,20])
print("bias의 갯수 : {}".format(len(bias)))

### (3) 합성곱 연산 진행하기

In [ ]:
# 합성곱 연산 진행하기
outputs = np.zeros([2,2,2])
for i in range(2):
    for j in range(2):
        for k in range(2):
            result = np.sum(inputs[i:i+3,j:j+3,k] * filters[k])
            outputs[i,j,k] = result
outputs

In [ ]:
# bias를 더해줌
outputs = outputs + bias 
outputs

<br>

## 2. 합성곱 연산 후 활성화함수 추가하기
---

* 이전의 DNN 구조와 동일하게 합성곱을 거친 후, 활성화함수를 거침으로써,<br>
보다 복잡한 특징들을 추출할 수 있게 됩니다.

In [ ]:
def relu(x):
    return np.maximum(x,0)

In [ ]:
# 활성화 함수를 거침
a = relu(outputs)
a

합성곱 신경망의 한 계층은 DNN과 같이 두단계를 거칩니다.<br>
1. 합성곱 연산(with bias)
2. 활성화 함수

<br>

## 3. 합성곱 층의 Notation
---

* 이후 논문과 모델 구조를 보면서 빠르게 이해할 수 있도록, 주요 Notation을 정리하겠습니다.

$
f^{[l]} = \mbox{filter size} \\
p^{[l]} = \mbox{padding} \\
s^{[l]} = \mbox{stride} \\
k^{[l]} = \mbox{number of filters}\\
----\\
\mbox{input shape : } (n_h^{[l-1]},n_w^{[l-1]},n_c^{[l-1]}) \\
\mbox{filter shape : } ( f^{[l]}_h,f^{[l]}_w,n_c^{[l-1]},k^{[l]}) \\
\mbox{output shape : } (n_h^{[l]},n_w^{[l]},n_c^{[l]})
$

### (1) 출력 층의 크기 (output shape)

출력 층의 높이와 폭은 아래의 수식 구조를 따릅니다.

$
n_h^{[l]} = \lfloor \frac{n_h^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$<br>
$
n_w^{[l]} = \lfloor \frac{n_w^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$


### (2) 출력 값의 채널 수

출력 층의 채널의 수($n_c^{[l]}$)는 필터의 갯수($k^{[l]}$)와 동일합니다.

$
n_c^{[l]} = k^{[l]}
$

### (3) 파라미터의 수 

각 합성곱 층은 Filter 내 Weight와 Bias로 이루어져 있습니다.<br>
$
\mbox{# parameter} = \mbox{#weights} + \mbox{#bias} \\
\mbox{# weights} = f_h^{[l]} * f_w^{[l]} * n_c^{l-1} * k^{[l]} \\
\mbox{# bias} = n_c^{[l]}
$

<br><br>

# \[ 2. CNN 모델 분석하기 \]

----
----
 
> *고전 모델 중 하나인 LeNet-5을 각 단계별로 출력 값의 크기 및 파라미터의 수를 계산해보도록 하겠습니다.*<br>


간단한 CNN 모델의 구성을 통해 계산해보기
----
![Imgur](https://i.imgur.com/DHpS6r8.png)


각 층 별 필터의 크기, 스트라이드, 패딩은 아래와 같습니다.

| Layer | # filters | filter size | stride | padding |
| ----- | -------   | ------ | ----- | ----- |
|  C1   | 6 | (5,5) | 1 | 0 |
|  S2   | 6 | (2,2) | 2 | 0 |
|  C3   | 16 | (5,5) | 1 | 0 |
|  S4   | 16 | (2,2) | 2 | 0 |
|  C5   | 120 | (5,5) | 1 | 0 |
|  F6   | 84 | --- | --- | --- |
|  OUTPUT   | 10 | --- | --- | --- |

## 1. 각 층 별 출력 층의 크기 계산하기
----

출력층은 아래의 수식을 따릅니다.<br>
$
n_h^{[l]} = \lfloor \frac{n_h^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$<br>
$
n_w^{[l]} = \lfloor \frac{n_w^{[l-1]}+2p^{[l]}-f_h^{[l]}}{s^{[l]}}+1\rfloor
$



#### 예제) 각 층 별 출력 층의 크기 계산하기 

| Layer | FEATURE MAP SIZE |
| ----- | -------   | 
| INPUT | (32,32,1) |
|  C1   | ? |
|  S2   | ? |
|  C3   | ? |
|  S4   | ? |
|  C5   | ? |
|  F6   | ? |
|OUTPUT | ? |

#### 정답 :

| Layer | FEATURE MAP SIZE |
| ----- | -------   | 
| INPUT | (32,32,1) |
|  C1   | (28,28,6) |
|  S2   | (14,14,6) |
|  C3   | (10,10,16) |
|  S4   | (5,5,16) |
|  C5   | (1,1,120)|
|  F6   | (84,) |
|  OUTPUT   | (10,) |

## 2. 각 층 별 파라미터의 수 계산하기
----

각 합성곱 층은 Filter 내 Weight와 Bias로 이루어져 있습니다.<br>
$
\mbox{#parameter} = \mbox{#weights} + \mbox{#bias} \\
\mbox{#weights} = f_h^{[l]} * f_w^{[l]} * n_c^{l-1} * k^{[l]} \\
\mbox{#bias} = n_c^{[l]}
$<br>
풀링층은 별도로 학습하는 파라미터가 없습니다.

#### Caution
* sub-sampling은 Max-Pooling으로 생각하고 계산해 주세요.

#### 예제) 각 층 별 출력 층의 크기 계산하기 

| Layer | # Parameter |
| ----- | -------   | 
| INPUT | 0 |
|  C1   | $(5*5*1*6) + 6 = 156 $ |
|  S2   | ? |
|  C3   | ? |
|  S4   | ? |
|  C5   | ? |
|  F6   | ? |
|OUTPUT | ?|

#### 정답 :

| Layer | # Parameter |
| ----- | -------   | 
| INPUT | 0 |
|  C1   | $(5*5*1*6) + 6 = 156 $ |
|  S2   | $0$ |
|  C3   | $(5*5*6*16) + 16 = 2,416 $ |
|  S4   | $0$ |
|  C5   | $(5*5*16*120)+120 = 48,120$ |
|  F6   | $120*84 + 84 = 10,164 $ |
|OUTPUT | $84*10 + 10 = 850$ |

# \[ 3.간단한 Convolution Neural Network 만들기 \]

----
----
 
> *자신만의 Convolution Neural Network 을 생성해보고 결과를 확인해 봅니다.*<br>


In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Input, Reshape, Flatten, Dense
from tensorflow.keras.models import Model
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.keras.utils import to_categorical
import tensorflow as tf 


# Parse images and labels
(images_train, labels_train) = data_train
(images_test, labels_test) = data_test
images_train = images_train/255.
images_test = images_test/255.
labels_train = to_categorical(labels_train, 10)
labels_test = to_categorical(labels_test, 10)


In [ ]:
inputs = Input(shape=(28,28))
res_inputs = Reshape((28, 28, 1))(inputs)

# layer 1 
conv_1 = Conv2D(6, 5, 1, 'same', activation='relu')(res_inputs)
mxpl_1 = MaxPooling2D(2, 2,'valid')(conv_1)

#layer 2
conv_2 = Conv2D(16, 5, 1, 'same',activation='relu')(mxpl_1)
mxpl_2 = MaxPooling2D(2, 2,'valid')(conv_2)

# layer 3 
conv_3 = Conv2D(120, 5, 1, 'same',activation='relu')(mxpl_2)
mxpl_3 = MaxPooling2D(2, 2,'valid')(conv_3)


# FC Layer 

flat_conv = Flatten()(mxpl_3)
fc1 = Dense(120, activation='relu', kernel_initializer='he_normal')(flat_conv)
logits = Dense(10, activation='softmax', kernel_initializer='glorot_normal')(fc1)


# Model 
model = Model(inputs, logits)
model.compile('adam', loss='categorical_crossentropy', metrics=['acc'])
model.summary()

In [ ]:
model.fit(x=images_train, y=labels_train, batch_size=64)

In [ ]:
loss, acc =  = model.evaluate(x=images_test, y=labels_test, batch_size=64, verbose=1)

---
⊙ Copyright(c) 2020 by PublicAI. All rights reserved <br>
All pictures, codes, writings cannot be copied without permission. <br>
Writen by PAI(info@publicai.co.kr) <br>
last updated on 2020/01/4 <br>

---